<a href="https://colab.research.google.com/github/lemon-mint20/Bachelorarbeit/blob/master/Bachelorarbeit_v5_2_Experiment_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Spectral Clustering – eine empirische Untersuchung** 

# Implementierung für Experiment 2


## Freie wissenschaftliche Arbeit zur Erlangung des akademischen Grades Bachelor of Science"

Studiengang: Wirtschaftsinformatik

**an der Wirtschaftswissenschaftlichen Fakultät der Universität Augsburg**

Lehrstuhl für Statistik

Eingereicht bei: Prof. Dr. Yarema Okhrin

Betreuerin:      Christine Distler (M. Sc.)

Vorgelegt von:

Adresse:         
>
>
>



Augsburg, im März 2023

# Wine Data Set
@misc{Dua:2019 ,
author = "Dua, Dheeru and Graff, Casey",
year = "2017",
title = "{UCI} Machine Learning Repository",
url = "http://archive.ics.uci.edu/ml",
institution = "University of California, Irvine, School of Information and Computer Sciences" }

In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from sklearn.datasets import load_wine
df, ground_truth = load_wine(return_X_y=True, as_frame=True)
df.head()

## Datenvorbereitung

In [ ]:
display(df.info())
display(df.describe())

In [ ]:
# The variables are measured in different units, so I rescale them to the same range
# standardize columns with min-max scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
display(df_scaled.head())

**veränderten Datensatz abspeichern, falls noch nicht vorhanden**

In [ ]:
# Speichere den veränderten Datensatz ab
# save Dataframe as csv in Google Drive
# from google.colab import drive
# drive.mount('drive')
# Write the DataFrame to CSV file.
# with open('/content/drive/My Drive/Colab Notebooks/Bachelorarbeit/wine-clustering_final.csv', 'w') as f:
#    df_scaled.to_csv(f)

In [ ]:
#save df_final as csv in local directory
# df_scaled.to_csv('wine-clustering_final.csv')

## Descriptive Statistik

In [ ]:
display(df_scaled.describe())
df_scaled.boxplot(figsize=(20,10))

In [ ]:
import seaborn as sns
sns.pairplot(df_scaled, diag_kind='kde', corner=True)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
corr = df_scaled.corr()
plt.figure(figsize=(15, 15))
sns.heatmap(corr, cmap='coolwarm', annot=True)
plt.show()

## Datensatz visualisieren 

In [ ]:

"""
https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60
Zugriff: 05.01.2023
"""

# Standardize the Data

from sklearn.preprocessing import StandardScaler

# Separating out the features
x = df_scaled.loc[:, :].values

# Standardizing the features
x = StandardScaler().fit_transform(x)


# PCA Projection to 2D
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents, 
                           columns = ['principal component 1', 'principal component 2'])


# Visualize 2D Projection
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.tick_params(labelsize=14)
ax.set_title('2 component PCA', fontsize = 20)
ax.scatter(principalDf['principal component 1'], principalDf['principal component 2'])
ax.grid()

# Gridsearch


## Ähnlichkeiten berechnen
Für die variable $\epsilon$ untersuche die Distanzen im Datensatz. Es muss sichergestellt werden, dass der $\epsilon$-Nachbarschaftsgraph verbunden ist. Die Distanzen können bei sich bei unterschiedlichen Distanzmaßen unterscheiden. Ich untersuche die Distanzen bei Distanzen 'euclidean' und 'correlation'

### Korrelations Distanz

In [ ]:
from scipy.spatial.distance import pdist, squareform

In [ ]:
distance = pdist(df_scaled, metric='correlation')
flatten_array_corr = np.array(distance).flatten()

plt.figure(figsize=(9, 5))
plt.hist(flatten_array_corr, bins=50)
plt.title("Korrelations Distanz", fontsize = 20)
plt.tick_params(labelsize=14)
plt.show()
df_dist_corr = pd.DataFrame(flatten_array_corr)
display(df_dist_corr.describe())
print("unique distances and their counts: ")
display(np.unique(flatten_array_corr, return_counts=True))
print(f"number of unique distances: {len(np.unique(flatten_array_corr))}")
print("distance matrix: ")
display(distance)
print(distance.shape)

### Euklidische Distanz

In [ ]:
distance = pdist(df_scaled, metric='euclidean')
flatten_array_eucl = np.array(distance).flatten()

plt.figure(figsize=(9, 5))
plt.hist(flatten_array_eucl, bins=50)
plt.title("Euklidische Distanz", fontsize = 20)
plt.tick_params(labelsize=14)
plt.show()

df_dist_eucl = pd.DataFrame(flatten_array_corr)
display(df_dist_eucl.describe())
print("unique distances and their counts: ")
display(np.unique(flatten_array_eucl, return_counts=True))
print(f"number of unique distances: {len(np.unique(flatten_array_eucl))}")
print("distance matrix: ")
display(distance)
print(distance.shape)

### $sigma$-Grenzen finden

In [ ]:
# für euklidische Distanz
def gaussian(mat, sigma):
    return np.exp(-np.square(mat) / (2 * sigma)**2)
dist_eucl_unique = np.unique(flatten_array_eucl)

import matplotlib.pyplot as plt
plt.figure(figsize=(7,7))
plt.scatter(dist_eucl_unique, gaussian(dist_eucl_unique, 1))
plt.scatter(dist_eucl_unique, gaussian(dist_eucl_unique, 0.2))
plt.xlabel('Euklidische Distanz', fontsize=12)
plt.ylabel('Gaußsche Ähnlichkeitsfunktion', fontsize=12)
plt.tick_params(labelsize=14)
plt.legend(['sigma=1', 'sigma=0.2'])
plt.show()

In [ ]:
# für corr
dist_corr_unique = np.unique(flatten_array_corr)
plt.figure(figsize=(7,7))
plt.scatter(dist_corr_unique, gaussian(dist_corr_unique, 1))
plt.scatter(dist_corr_unique, gaussian(dist_corr_unique, 0.2))
plt.xlabel('Korrelationsdistanz', fontsize=12)
plt.ylabel('Gaußsche Ähnlichkeitsfunktion', fontsize=12)
plt.tick_params(labelsize=14)
plt.legend(['sigma=1', 'sigma=0.2'])
plt.show()

### optimale Clusterzahl schätzen (Eigengap Heuristik)

In [ ]:
sigma = 0.5
Adj_mat = squareform(pdist(df_scaled, metric='euclidean'))
W = np.exp(-np.square(Adj_mat) / (2 * sigma)**2)
D = np.diag(np.sum(W, axis=1))
L = D - W
D_inv = np.diag(1 / np.diag(D))
L_rw = np.dot(D_inv, L)
print(L_rw.round(2))

In [ ]:
eigenvalues, eigenvectors = np.linalg.eig(np.dot(D_inv, L))

In [ ]:
# print(np.argsort(np.diff(eigenvalues)))
# print(f'Eigenvalues: {eigenvalues}')
def maxDiffIndex(array): 
  array = np.sort(array) 
  max_diff_index = 0
  max_diff = 0
  
  for i in range(1, len(array)): 
    diff = array[i] - array[i-1]
    if diff > max_diff:
      max_diff_index = i
      max_diff = diff
  
  return max_diff_index, max_diff
print(maxDiffIndex(eigenvalues))

In [ ]:
plt.figure(figsize=(12,5))
plt.scatter([i for i in range(len(eigenvalues))], np.sort(eigenvalues))
plt.show()

### Parameter-Grid initialisieren

In [ ]:
from sklearn.model_selection import ParameterGrid
params1 = {
    'K': [3],
    # more metric arguments from scipy.spatial.distance.pdist
    'metric': ['euclidean', 'correlation'],
    'sim_graph': ['fully_connect'],
    'normalized': [1, 2, 3],
    'sigma': np.linspace(0.2, 1, 9),
}

grid1 = list(ParameterGrid(params1))
print(len(grid1))

params2_1 = {
    'K': [3],
    # more metric arguments from scipy.spatial.distance.pdist
    'metric': ['euclidean'],
    'sim_graph': ['eps_neighbor'],
    'normalized': [1, 2, 3],
    'epsi': np.linspace(df_dist_eucl.quantile(0.20), df_dist_eucl.quantile(0.8), 40),
}

grid2_1 = list(ParameterGrid(params2_1))
print(len(grid2_1))

params2_2 = {
    'K': [3],
    # more metric arguments from scipy.spatial.distance.pdist
    'metric': ['correlation'],
    'sim_graph': ['eps_neighbor'],
    'normalized': [1, 2, 3],
    'epsi': np.linspace(df_dist_corr.quantile(0.20), df_dist_corr.quantile(0.8), 20),
}

grid2_2 = list(ParameterGrid(params2_2))
print(len(grid2_2))

params3 = {
    'K': [3],
    # more metric arguments from scipy.spatial.distance.pdist
    'metric': ['euclidean', 'correlation'],
    'sim_graph': ['knn'],
    'normalized': [1, 2, 3],
    'knn': np.arange(3, 20),
}

grid3 = list(ParameterGrid(params3))
print(len(grid3))

params4 = {
    'K': [3],
    # more metric arguments from scipy.spatial.distance.pdist
    'metric': ['euclidean', 'correlation'],
    'sim_graph': ['mutual_knn'],
    'normalized': [1, 2, 3],
    'knn': np.arange(3, 20),
}

grid4 = list(ParameterGrid(params4))
print(len(grid4))

grid = grid1+grid2_1+grid2_2+grid3+grid4
print(len(grid))


# Implementierung des Spectral Clustering von Dr. Yikun Zhang 

Quelle: [https://github.com/zhangyk8/Spectral-Clustering/blob/master/spectral_clustering.py](https://github.com/zhangyk8/Spectral-Clustering/blob/master/spectral_clustering.py)

(Zugriff: 15.12.2023)

Die Berechnung für den vollverbundenen Graph wurde von mir abgeändert zu: 
```
W = np.exp(-np.square(Adj_mat) / (2 * sigma)**2)
```

In [ ]:
from sklearn.cluster import KMeans

# Based on "A Tutorial on Spectral Clustering" written by Ulrike von Luxburg
def Spectral_Clustering(X, K=8, adj=True, metric='euclidean', sim_graph='fully_connect', sigma=1.0, knn=10, epsi=0.5,
                        normalized=1):
    """
    Input:
        X : [n_samples, n_samples] numpy array if adj=True, or, a [n_samples_a, n_features] array otherwise;

        K: int, The number of clusters;

        adj: boolean, Indicating whether the adjacency matrix is pre-computed. Default: True;

        metric: string, A parameter passing to "scipy.spatial.distance.pdist()" function for computing the adjacency
        matrix (deprecated if adj=True). Default: 'euclidean';

        sim_graph: string, Specifying the type of similarity graphs. Choices are ['fully_connect', 'eps_neighbor',
        'knn', 'mutual_knn']. Default: 'fully_connect';

        sigma: float, The variance for the Gaussian (aka RBF) kernel (Used when sim_graph='fully_connect'). Default: 1;

        knn: int, The number of neighbors used to construct k-Nearest Neighbor graphs (Used when sim_graph='knn'
        or 'mutual_knn'). Default: 10;

        epsi: float, A parameter controlling the connections between points (Used when sim_graph='eps_neighbor').
        Default: 0.5;

        normalized: int, 1: Random Walk normalized version; 2: Graph cut normalized version; other integer values:
        Unnormalized version. Default: 1.

    Output:
        sklearn.cluster class, Attributes:
            cluster_centers_ : array, [n_clusters, n_features], Coordinates of cluster centers in K-means;
            labels_ : Labels of each point;
            inertia_ : float, Sum of squared distances of samples to their closest cluster center in K-means;
            n_iter_ : int, Number of iterations run in K-means.
    """
    # Compute the adjacency matrix
    if not adj:
      Adj_mat = squareform(pdist(X, metric=metric))
    else:
        Adj_mat = X
    # Compute the weighted adjacency matrix based on the type of similarity graphs
    if sim_graph == 'fully_connect':
        W = np.exp(-np.square(Adj_mat) / (2 * sigma)**2)
    elif sim_graph == 'eps_neighbor':
        W = (Adj_mat <= epsi).astype('float64')
    elif sim_graph == 'knn':
        W = np.zeros(Adj_mat.shape)
        # Sort the adjacency matrx by rows and record the indices
        Adj_sort = np.argsort(Adj_mat, axis=1)
        # Set the weight (i,j) to 1 when either i or j is within the k-nearest neighbors of each other
        for i in range(Adj_sort.shape[0]):
            W[i, Adj_sort[i, :][:(knn + 1)]] = 1
    elif sim_graph == 'mutual_knn':
        W1 = np.zeros(Adj_mat.shape)
        # Sort the adjacency matrx by rows and record the indices
        Adj_sort = np.argsort(Adj_mat, axis=1)
        # Set the weight W1[i,j] to 0.5 when either i or j is within the k-nearest neighbors of each other (Flag)
        # Set the weight W1[i,j] to 1 when both i and j are within the k-nearest neighbors of each other
        for i in range(Adj_mat.shape[0]):
            for j in Adj_sort[i, :][:(knn + 1)]:
                if i == j:
                    W1[i, i] = 1
                elif W1[i, j] == 0 and W1[j, i] == 0:
                    W1[i, j] = 0.5
                else:
                    W1[i, j] = W1[j, i] = 1
        W = np.copy((W1 > 0.5).astype('float64'))
    else:
        raise ValueError(
            "The 'sim_graph' argument should be one of the strings, 'fully_connect', 'eps_neighbor', 'knn', or 'mutual_knn'!")

    # Compute the degree matrix and the unnormalized graph Laplacian
    D = np.diag(np.sum(W, axis=1))
    L = D - W

    # Compute the matrix with the first K eigenvectors as columns based on the normalized type of L
    if normalized == 1:  ## Random Walk normalized version
        # Compute the inverse of the diagonal matrix
        D_inv = np.diag(1 / np.diag(D))
        # Compute the eigenpairs of L_{rw}
        Lambdas, V = np.linalg.eig(np.dot(D_inv, L))
        # Sort the eigenvalues by their L2 norms and record the indices
        ind = np.argsort(np.linalg.norm(np.reshape(Lambdas, (1, len(Lambdas))), axis=0))
        V_K = np.real(V[:, ind[:K]])
    elif normalized == 2:  ## Graph cut normalized version
        # Compute the square root of the inverse of the diagonal matrix
        D_inv_sqrt = np.diag(1 / np.sqrt(np.diag(D)))
        # Compute the eigenpairs of L_{sym}
        Lambdas, V = np.linalg.eig(np.matmul(np.matmul(D_inv_sqrt, L), D_inv_sqrt))
        # Sort the eigenvalues by their L2 norms and record the indices
        ind = np.argsort(np.linalg.norm(np.reshape(Lambdas, (1, len(Lambdas))), axis=0))
        V_K = np.real(V[:, ind[:K]])
        if any(V_K.sum(axis=1) == 0):
            raise ValueError(
                "Can't normalize the matrix with the first K eigenvectors as columns! Perhaps the number of clusters K or the number of neighbors in k-NN is too small.")
        # Normalize the row sums to have norm 1
        V_K = V_K / np.reshape(np.linalg.norm(V_K, axis=1), (V_K.shape[0], 1))
    else:  ## Unnormalized version
        # Compute the eigenpairs of L
        Lambdas, V = np.linalg.eig(L)
        # Sort the eigenvalues by their L2 norms and record the indices
        ind = np.argsort(np.linalg.norm(np.reshape(Lambdas, (1, len(Lambdas))), axis=0))
        V_K = np.real(V[:, ind[:K]])

    # Conduct K-Means on the matrix with the first K eigenvectors as columns
    kmeans = KMeans(n_clusters=K, init='k-means++', random_state=0).fit(V_K)
    return kmeans

# Spectral clustering auf alle Datensätze anwenden# run spectral clustering

In [ ]:
import time as time
import warnings
from sklearn.metrics import adjusted_rand_score, silhouette_score, calinski_harabasz_score, davies_bouldin_score

warnings.filterwarnings("error")

Im selbstgenerierten Datensatz sind die ground truth classes bekannt. Daher Werte ich das Clustering mit dem ARI aus. 

In [ ]:
def evaluate_single_dataset(dataset):
  silhouette_scores = []
  calinski_h_scores = []
  davies_b_scores = []
  ari = []

  has_exception = []
  iter_with_LinAlgError = []
  iter_with_ValueError = []
  iter_with_RuntimeWarning = []
  iter_with_Errors_Or_Warnings = []
  iter_without_Errors = []
  # eigenvalues_of_one_dataset = []
  
  dataset_index = []
  metric = []
  simgraph = []
  sigma = []
  knn = []
  epsi = []
  normalized = []

  for i in range(len(grid)):
    try:
      clust = Spectral_Clustering(X=dataset, **grid[i], adj=False)
    except np.linalg.LinAlgError:
      iter_with_LinAlgError.append(i)
      has_exception.append(1)
      continue
    except ValueError as e:
      # print(e)
      iter_with_ValueError.append(i)
      has_exception.append(1)
      continue
    except RuntimeWarning as e: 
      # print(e)
      iter_with_RuntimeWarning.append(i)
      has_exception.append(1)
      continue
    except:
      iter_with_Errors_Or_Warnings.append(i)
      has_exception.append(1)
      continue
    else:
      dataset_index.append(i)
      has_exception.append(0)
      metric.append(grid[i]['metric'])
      simgraph.append(grid[i]['sim_graph'])
      if 'knn' in grid[i]:
        knn.append(grid[i]['knn'])
        sigma.append(-1)
        epsi.append(-1)
      if 'sigma' in grid[i]:
        sigma.append(grid[i]['sigma'])
        knn.append(-1)
        epsi.append(-1)
      if 'epsi' in grid[i]:
        epsi.append(grid[i]['epsi'])
        knn.append(-1)
        sigma.append(-1)
      normalized.append(grid[i]['normalized'])
      ari.append(adjusted_rand_score(clust.labels_, ground_truth))

      iter_without_Errors.append(i)
      score1 = silhouette_score(dataset, clust.labels_)
      score2 = calinski_harabasz_score(dataset, clust.labels_)
      score3 = davies_bouldin_score(dataset, clust.labels_)
      silhouette_scores.append(score1)
      calinski_h_scores.append(score2)
      davies_b_scores.append(score3)
      # eigenvalues_of_one_dataset.append(eigenvalues)

      # print(f'\t Iteration: {i}')
      # print(f'silhouette:         {score1}')
      # print(f'calinski_harabasz:  {score2}')
      # print(f'davies_bouldin: 
      
  return (silhouette_scores, 
          calinski_h_scores, 
          davies_b_scores, 

          metric,
          simgraph,
          sigma,
          knn,
          epsi, 
          normalized,
          
          iter_with_LinAlgError, 
          iter_with_ValueError, 
          iter_with_RuntimeWarning, 
          iter_with_Errors_Or_Warnings, 
          iter_without_Errors, 
          # eigenvalues_of_one_dataset, 
          dataset_index,
          ari)

In [ ]:
startTime = time.time()

(silhouette_scores, 
calinski_h_scores, 
davies_b_scores, 
metric, 
simgraph,
sigma, 
knn, 
epsi, 
normalized, 
iter_with_LinAlgError, 
iter_with_ValueError, 
iter_with_RuntimeWarning, 
iter_with_Errors_Or_Warnings, 
iter_without_Errors, 
# eigenvalues_of_one_dataset, 
dataset_index, 
ari) = evaluate_single_dataset(df_scaled)

executionTime = (time.time() - startTime)
print('Execution time in seconds: ' + str(executionTime))

# Datensatz zur Auswertung

In [ ]:
import pandas as pd

In [ ]:
df_experiment2 = pd.DataFrame(data={
        'Dataset': dataset_index,
        'Iteration': iter_without_Errors,
        'metric': metric,
        'sim_graph': simgraph,
        'sigma': sigma,
        'knn': knn,
        'epsi': np.hstack(epsi),
        'normalised': normalized,
        'Calinski': np.hstack(calinski_h_scores),
        'Davis': np.hstack(davies_b_scores),
        'Silhouette': np.hstack(silhouette_scores),
        'ari': ari,
        })

In [ ]:
df_experiment2

In [ ]:
# save Dataframe as csv in Google Drive

from google.colab import drive
drive.mount('drive')
# Write the DataFrame to CSV file.
with open('/content/drive/My Drive/Colab Notebooks/Bachelorarbeit/Experiment2.csv', 'w') as f:
  df_experiment2.to_csv(f)

In [ ]:
df_experiment2.to_csv('Experiment2.csv')

# Datensatz importieren

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from google.colab import drive
# drive.mount('drive')
# df_experiment2 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Bachelorarbeit/Experiment2.csv')
df_experiment2 = pd.read_csv('Experiment2.csv')

In [ ]:
# Datensatz auf NaN-Werte überprüfen
display(df_experiment2.info())
df_experiment2.head()

In [ ]:
df_experiment2 = df_experiment2.drop(['Unnamed: 0', 'Dataset'], axis=1)

## ARI-Werte untersuchen

In [ ]:
# Verteilung der ARI-Werte untersuchen
# print(pd.unique(df.ari))
plt.figure(figsize=(10, 5))
plt.hist(df_experiment2.ari, bins=len(pd.unique(df_experiment2.ari)))
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
ari_rounded = df_experiment2.ari.round(1)
ari, counts = np.unique(ari_rounded, return_counts=True)
print(ari, counts)
print(counts/(len(ari_rounded)))

In [ ]:
df_experiment2.ari.describe()

In [ ]:
plt.figure(figsize=(20.0, 4.0))
plt.tick_params(labelsize=14)
plt.title('Adjusted Rand Index')
aris = df_experiment2.ari
plt.plot(aris, 'bo', aris, 'k')
# make more xticks
plt.xticks(np.arange(0, len(aris), 20))
plt.show()

## ARI für jede Iteration geplottet

In [ ]:
plt.figure(figsize=(20.0, 10.0))

plt.subplot(311)
plt.title('Silhouettenkoeffizient')
plt.plot(df_experiment2.Silhouette, 'bo', df_experiment2.Silhouette, 'k')
plt.xticks(np.arange(0, len(aris), 20))

plt.subplot(312)
plt.title('Calinski-Harabasz Index')
plt.plot(df_experiment2.Calinski, 'bo', df_experiment2.Calinski, 'k')
plt.xticks(np.arange(0, len(aris), 20))

plt.subplot(313)
plt.title('Davies-Bouldin Index')
plt.plot(df_experiment2.Davis, 'bo', df_experiment2.Davis, 'k')
plt.xticks(np.arange(0, len(aris), 20))

plt.show()

# Clusteringergebnisse untersuchen

## *1.* Untersuchung der Metrik auf die Clusterqualität

In [ ]:
df_experiment2.groupby('metric').ari.describe().round(4)

## *2.* Einfluss des Ähnlichkeitsgraphen auf die Clusterqualität

In [ ]:
df_experiment2.groupby('sim_graph').ari.describe().round(4)

## *3.* Einfluss von $\sigma$ auf die Clusterqualität
Der Einfluss soll mit der Korrelation berechnet werden. Um einzuschätzen, welches Korrelationsmaß verwendet werden soll und damit ich eine Einschätzung über die Richtung der Korrelation habe, plotte ich sigma und die Clustergüte in einem Scatterplot.


In [ ]:
import seaborn as sns

In [ ]:
plt.figure(figsize=(5, 5))
sns.scatterplot(data=df_experiment2[df_experiment2.sigma != -1], x='sigma', y='ari')
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
df_experiment2[df_experiment2.sigma != -1].sigma.corr(df_experiment2[df_experiment2.sigma != -1].ari, method='spearman')

In [ ]:
# Was sind die häufigsten Werte? 
ari_round_unique = np.unique(np.round(df_experiment2[df_experiment2.sigma != 1].ari,1), return_counts=True)
# sortiere ari_round nach Häufigkeit
ari_round_unique_sorted = np.array([x for _,x in sorted(zip(ari_round_unique[1],ari_round_unique[0]), reverse=True)])
ari_round_unique_sorted

In [ ]:
df2 = df_experiment2[df_experiment2.sigma != -1].copy()
df2.ari = np.round(df2.ari, 1)
df3 = df2[df2.ari.isin(ari_round_unique_sorted[:4])].copy()
# df3.groupby(['ari', 'metric']).sigma.describe()
# df3.groupby(['sigma']).ari.describe()
df3.groupby(['ari']).sigma.describe().round(4)

## *4.1* Einfluss von $k$ beim knn-Graphen auf die Clusterqualität

In [ ]:
# plotte die Spalte k, bei dem der sim-graph knn ist und die Clustergüte in einem Scatterplot
plt.figure(figsize=(7, 7))
sns.scatterplot(data=df_experiment2[df_experiment2.sim_graph == 'knn'], x='knn', y='ari')
plt.tick_params(labelsize=14)
plt.show()

## *4.2* Einfluss von $k$ beim mutual-knn-Graphen auf die Clusterqualität

In [ ]:
plt.figure(figsize=(7, 7))
sns.scatterplot(data=df_experiment2[df_experiment2.sim_graph == 'mutual_knn'], x='knn', y='ari')
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
# Wie viele Iterationen wurden insgesammt mit dem mutual-knn erstellt.
df_experiment2[df_experiment2.sim_graph == 'mutual_knn'] 
# df[(df.sim_graph == 'mutual_knn') & (df.ari != 1.0) & (df.ari != 0.0)]

In [ ]:
df_experiment2[(df_experiment2.normalised == 2) & (df_experiment2.sim_graph == 'mutual_knn')]

## *5.* Einfluss von $\epsilon$ auf die Clusterqualität

In [ ]:
plt.figure(figsize=(15, 5))
sns.scatterplot(data=df_experiment2[df_experiment2.sim_graph == 'eps_neighbor'], x='epsi', y='ari')
plt.tick_params(labelsize=14)
plt.show()

In [ ]:
df_experiment2[df_experiment2.sim_graph == 'eps_neighbor']

## *6.* Einfluss vom Laplace-Graphen auf die Clusterqualität

In [ ]:
df_experiment2.groupby('normalised').ari.describe().round(4)

In [ ]:
plt.figure(figsize=(10, 5))
sns.boxplot(data=df_experiment2, x='normalised', y='ari')
plt.tick_params(labelsize=14)
plt.show()

# Zusammenfassung

In [ ]:
df_experiment2.head()

In [ ]:
a = df_experiment2.groupby(['metric']).ari.describe().round(4)
b = df_experiment2.groupby(['sim_graph']).ari.describe().round(4)
c = df_experiment2[df_experiment2.sim_graph == 'fully_connect'].groupby(['metric']).ari.describe().round(4)
c.index = c.index + '_fully_connect'
d = df_experiment2.groupby(['normalised']).ari.describe().round(4)

frames = [a,b,c,d]
summary = pd.concat(frames)
summary = summary.rename({"metric":"parameter"})

In [ ]:
summary.sort_values(by=['mean','std'], ascending=[False, True])[['mean', 'std']]

In [ ]:
df_experiment2.groupby(['sim_graph', 'metric', 'normalised']).ari.describe().round(4).sort_values(by=['mean','std'], ascending=[False, True])[['mean', 'std']]